## Goal - Notebook Description¶

- Read genomic mutations from MelanoDB
- Create preprocessed Table
- Create simple vizualisation (heatmap)

In [1]:
# imports
import time
import pandas as pd
import numpy as np
import seaborn as sns
import json
import sqlite3
import os
import re
import matplotlib.pyplot as plt
import ast
from joblib import Parallel, delayed

In [2]:
# Load MelanoDB
cnx = sqlite3.connect('../database/MelanoDB.db')
df_snv =  pd.read_sql_query("SELECT * FROM snvs", cnx)

In [3]:
#Replace all sources text by only the author name
for i in range(0,len(df_snv)):
    df_snv['source'] = df_snv['source'].apply(lambda x : ast.literal_eval(x)['author'])

SyntaxError: invalid syntax (<unknown>, line 1)

## Fetch list of genes from sources

In [4]:
sources_to_keep = ['Pauline Blateau, Jerome Solassol', 'Federica Catalanotti, David B. Solit', 'Eliezer M. Van Allen, Dirk Schadendorf', 'Baptiste Louveau, Samia Mourah']

In [5]:
dict_gene_sources = {}
for source in sources_to_keep:
    numb_genes = list(df_snv[df_snv['source']==source]['HGNC'].unique())
    print(f'{source}: {len(numb_genes)}')
    if numb_genes !=0:
        dict_gene_sources[source]=numb_genes

Pauline Blateau, Jerome Solassol: 35
Federica Catalanotti, David B. Solit: 218
Eliezer M. Van Allen, Dirk Schadendorf: 13383
Baptiste Louveau, Samia Mourah: 18


In [6]:
list_genes_blateau = dict_gene_sources['Pauline Blateau, Jerome Solassol']
list_allen = dict_gene_sources['Eliezer M. Van Allen, Dirk Schadendorf']

In [7]:
string_to_change = "ABL1 GNAQ RAC1 AKT1 GRIN2A RAF1 AKT2 GRM3 RASA2 AKT3 HOXD8 RET ALK HRAS RPS27 ARAF IDH1 SERPINB3 BRAF IGF1R SNX31 BTK JAK2 STAT3 CCND1 KDR STK11 CDK4 KIT STK19 CDKN2A KRAS TACC1 CD274 MAP2K1 TERT CTNNB1 MAP2K2 TRRAP CXCR4 MDM2 WT1 DDR1 MET DDR2 MITF DDX3X MRPS31 DUSP4 NOTCH1 EGFR NOTCH2 ERBB2 NRAS ERBB3 PDGFRA ERBB4 PDGFRB EZH2 PHLPP1 FBXW7 PIK3CA FERMT3 PIK3CG FGFR1 PIK3R1 FGFR2 PIK3R2 FGFR3 PLCG2 FLT3 PPP6C GNA11 PTPN11"
string_to_change = string_to_change.replace(' ', ',')
list_louveau = string_to_change.split(',')

In [8]:
list_catalanotti = "ABL1,ABL2,AKT1,AKT2,AKT3,ALK,ALOX12B,APC,AR,ARAF,ARHGAP26,ARID1A,ARID2,ASXL1,ATM,ATRX,AURKA,BAP1,BCL2L1,BCL2L11,BCL6,BCOR,BIRC2,BRAF,BRCA1,BRCA2,BUB1B,CARD11,CBL,CBLB,CBLC,CCND1,CCNE1,CD79B,CDC42EP2,CDC73,CDH1,CDH11,CDK12,CDK4,CDK6,CDK8,CDKN1A,CDKN2A,CDKN2B,CDKN2C,CEBPA,CHEK1,CHEK2,CIC,CREBBP,CRKL,CRLF2,CSF1R,CTNNB1,CYLD,DAXX,DDR2,DICER1,DIS3,DNMT1,DNMT3A,DNMT3B,E2F3,EGFR,EIF4EBP1,EP300,EPHA10,EPHA2,EPHA3,EPHA4,EPHA5,EPHA6,EPHA7,EPHA8,EPHB1,EPHB2,EPHB3,EPHB4,EPHB6,ERBB2,ERBB3,ERBB4,ERG,ESR1,ETV1,ETV6,EZH2,FAM123B,FAM46C,FAS,FAT1,FBXO11,FBXW7,FGFR1,FGFR2,FGFR3,FGFR4,FH,FLCN,FLT1,FLT3,FLT4,FOXL2,FUBP1,GATA1,GATA2,GATA3,GLI1,GLI3,GNA11,GNAQ,GNAS,GOLPH3,GRIN2A,GRM3,GSK3B,HDAC2,HIF1A,HMGA2,HNF1A,HRAS,HSP90AA1,IDH1,IDH2,IGF1R,IGFBP7,IKBKE,IKZF1,IL7R,INPP4A,INPP4B,INSR,IRS1,IRS2,JAK1,JAK2,JAK3,JUN,KCNJ5,KDM5C,KDM6A,KDR,KEAP1,KIT,KLF6,KRAS,LDHA,LGR6,LMO1,MAGI2,MAP2K1,MAP2K2,MAP2K4,MAP3K1,MAP3K8,MCL1,MDM2,MDM4,MED12,MEF2B,MEN1,MET,MITF,MLH1,MLL,MLL2,MLL3,MLST8,MPL,MSH2,MSH6,MTOR,MYB,MYC,MYCL1,MYCN,MYD88,NCOA2,NF1,NF2,NFE2L2,NFKB1,NFKB2,NKX2-1,NOTCH1,NOTCH2,NOTCH3,NOTCH4,NPM1,NRAS,NTRK1,NTRK2,NTRK3,PAK7,PALB2,PARK2,PARP1,PAX5,PBRM1,PDGFRA,PDGFRB,PHOX2B,PIK3C2G,PIK3CA,PIK3CB,PIK3CD,PIK3CG,PIK3R1,PIK3R2,PIK3R3,PKM2,PLK2,PNRC1,PPP2R1A,PRDM1,PREX2,PRKAA2,PRKAR1A,PRKCI,PTCH1,PTEN,PTPN11,PTPRD,PTPRS,PTPRT,RAF1,RARA,RB1,REL,RET,RICTOR,RNF43,ROR2,ROS1,RPTOR,RUNX1,SDHB,SETD2,SF3B1,SHQ1,SMAD2,SMAD3,SMAD4,SMARCA4,SMARCB1,SMO,SOCS1,SOX2,SPOP,SRC,SRSF2,STAG2,STK11,SUFU,TBK1,TEK,TERT,TET1,TET2,TGFBR2,TMPRSS2,TNFAIP3,TNFRSF14,TOP1,TP53,TP63,TSC1,TSC2,TSHR,U2AF1,VHL,WAS,WNK1,WT1,XPO1,YAP1,YES1,ZRSR2"
list_catalanotti = string_to_change.split(',')

In [9]:
list_seq_genes_per_sources = [list_genes_blateau, list_allen, list_louveau, list_catalanotti]

In [10]:
list_hgnc = list(set.intersection(*map(set,list_seq_genes_per_sources)))
list_hgnc

['RAC1',
 'STK11',
 'PIK3CA',
 'PDGFRA',
 'ERBB4',
 'BRAF',
 'FGFR3',
 'NRAS',
 'ALK',
 'DDR2',
 'AKT1',
 'KIT',
 'JAK2',
 'MET',
 'FGFR1',
 'EGFR',
 'CDKN2A',
 'GNA11',
 'ERBB2',
 'CTNNB1',
 'TERT',
 'FGFR2',
 'MAP2K1']

## Create mutation table

In [12]:
df_mutations_onco = df_snv.copy()

In [13]:
df_mutations_onco = df_mutations_onco[df_mutations_onco.HGNC.isin(list_hgnc)]
df_mutations_onco.shape

(1569, 22)

In [14]:
list_patients = list(df_mutations_onco['patientID'].unique())
print(len(list_patients))

185


In [15]:
dataframe_boolean_mut = pd.DataFrame()
dataframe_boolean_mut.insert(0, 'patientID', list_patients)
for hgnc in list_hgnc:
    sub_tab = df_mutations_onco[df_mutations_onco['HGNC']==hgnc]
    sub_tab = sub_tab.drop_duplicates(['patientID','HGNC'])
    list_pat_mut = sub_tab['patientID'].unique()
    list_not_pat_mut = [item for item in list_patients if item not in list_pat_mut]
    
    list_append_pat_id = []
    list_append_mut_value = []
    for i in range(0, len(sub_tab)):
        if(sub_tab['mutated'].values[i]=='no'):
            list_append_pat_id.append(sub_tab['patientID'].values[i])
            list_append_mut_value.append(int(0))
        elif(sub_tab['mutated'].values[i]=='yes'):
            list_append_pat_id.append(sub_tab['patientID'].values[i])
            list_append_mut_value.append(int(1))
    for i in range(0,len(list_not_pat_mut)):
        list_append_pat_id.append(list_not_pat_mut[i])
        list_append_mut_value.append(int(0))
    
    df_temp = pd.DataFrame({'patientID': list_append_pat_id, 
                            hgnc: list_append_mut_value})
    dataframe_boolean_mut = dataframe_boolean_mut.merge(right = df_temp, on='patientID')
    dataframe_boolean_mut[hgnc] = dataframe_boolean_mut[hgnc].astype('Int64')

In [16]:
dataframe_boolean_mut

,patientID,RAC1,STK11,PIK3CA,PDGFRA,ERBB4,BRAF,FGFR3,NRAS,ALK,...,MET,FGFR1,EGFR,CDKN2A,GNA11,ERBB2,CTNNB1,TERT,FGFR2,MAP2K1
0,BS_000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,BS_001,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,BS_002,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,BS_003,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BS_004,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,LM_22,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
181,LM_23,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
182,LM_25,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183,LM_26,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#set first column as index to be able to transpose
dataframe_boolean_mut = dataframe_boolean_mut.set_index('patientID')
transpose_boolean_mut = dataframe_boolean_mut.T
transpose_boolean_mut.head()

patientID,BS_000,BS_001,BS_002,BS_003,BS_004,BS_005,BS_006,BS_007,BS_008,BS_009,...,LM_17,LM_18,LM_19,LM_20,LM_21,LM_22,LM_23,LM_25,LM_26,LM_27
RAC1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
STK11,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PIK3CA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PDGFRA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERBB4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
transpose_boolean_mut.to_csv("./inputs/melanodb_snv.csv", sep=',', header=True, index=True)

### Get metadata info - patient-id & sources

In [19]:
df_snv_annotations_sources = df_snv[['patientID', 'source']]

In [20]:
df_snv_annotations_sources = df_snv_annotations_sources.drop_duplicates().set_index('patientID')

In [21]:
df_snv_annotations_sources.to_csv("./inputs/melanodb_snv_infos.csv", sep=',', header=True, index=True)